# Overview

In this notebook we introduce some basic functionality of the custom `cnn` library.

## Choosing GPU

Recall that by default Tensorflow will use every free GPU available. Use the following code to choose one specific GPU to run your code. Also, after you are finished, to release your current GPU, make sure that the current iPython kernel is closed (File > Close & Halt, or Kernel > Shutdown).

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER']= 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']= '0'
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## Import

The following libraries will be used in this tutorial. The custom montage module is a pylab wrapper for quickly visualizing 3D grayscale images.

In [ ]:
import cnn
import numpy as np
from montage import montage

# Client Class

The `cnn.utils.Client` class is used to interact with underlying database images. As needed, the `cnn.db.Manager()` class can be used to interact with the MongoDB database.

In [ ]:
# Initializing a Client class
app_context = {
    'name': 'brats',
    'ip_mongodb': '160.87.25.45',
    'tags-series': ['brats-t2'],
    'tags-labels': ['tumor']
}

client = cnn.utils.Client(app_context)

In [ ]:
# Initializing a Manager class

man = cnn.db.Manager(ip='160.87.25.45')
man.use()

In [ ]:
def test_client():
    
    infos = {
        'shape': [1, 240, 240],
        'tiles': [1, 0, 0]
    }

    # vol['dat'] contains data
    # vol['lbl'] contains label(s)
    vol = client.load_next(random=True, infos=infos)

    # Determine tumor presence based on max label
    tumor = np.max(vol['lbl'])
    tumor = 'tumor' if tumor > 0 else 'no tumor'
    print(tumor)
    montage(vol['dat'][..., 0])
    montage(vol['lbl'])
    print(vol['dat'].shape)

## Overloading Client

Recall in object-oriented programming, template objects can be overloaded with custom methods (and variables) based on a specific task. Here we use the templte Client class to create our own unique Client to handle data for our specific experiment.

In [ ]:
class Client(cnn.utils.Client):
    
    def init_client(self):
        """
        Method to set default experiment specific variables
        
        """
        self.infos = {
            'shape': [1, 240, 240],
            'tiles': [1, 0, 0]
        }
        
        self.inputs = {
            'dtypes': {},
            'shapes': {},
            'classes': {}
        }
        
        self.inputs['dtypes']['dat'] = 'float32'
        self.inputs['shapes']['dat'] = [1, 240, 240, 1]
        
        self.inputs['dtypes']['sce-tumor'] = 'int32'
        self.inputs['shapes']['sce-tumor'] = [1, 1, 1, 1]
        self.inputs['classes']['sce-tumor'] = 2 
        
        self.dist = {
            0: 0.50, 
            1: 0.125, 
            2: 0.125, 
            3: 0.125, 
            4: 0.125}
        
        self.mode = 'mixed'
        self.iids = {}
        self.x = {}
        
    def get(self, mode=None, random=True):
        """
        Method to load a single train/valid study
        
        """
        # Load data
        next_doc = self.next_doc(mode=mode, random=True)
        vol = self.load(doc=next_doc['doc'], infos=next_doc['infos'])
        
        # Preprocessing
        if vol['dat'].any():
            vol['dat'] = (vol['dat'] - np.mean(vol['dat'])) / np.std(vol['dat'])
        else:
            vol['dat'][:] = -1
        
        # Manually convert mask labels to classification labels
        tumor = np.max(vol['lbl'])
        tumor = 1 if tumor == 0 else 2
        vol['lbl'] = np.array(tumor).reshape(1, 1, 1, 1)
        
        # Create nested vol dictionary
        vol['lbl'] = {'sce-tumor': vol['lbl']}
        
        return self.return_get(next_doc, vol)

In [ ]:
def test_get():

    client = Client(
        app_context=app_context,
        # threads=2, 
        # batch=16,
        # capacity=128
    )
    client.prepare(fold=-1)
    dats = []
    
    for i in range(256):
        
        print('Loading study %03i' % i, end='\r')
        output = client.get_async(mode='mixed')
        output = client.create_inputs_dict(output, mode='train')
        dats.append(output['dat'])
    
    montage(np.concatenate(dats), vm=[-1, 5])

test_get()

# Model Class

The `cnn.Model` class is used to define our neural network model (graph), in addition to all our training hyperparameters. 

In [ ]:
class Model(cnn.ClassicModel):

    def init_hyperparams_custom(self):

        self.params['save_dir'] = fdir 
        self.params['batch_size'] = 16 
        self.params['iterations'] = 50 

        self.params['lnet_fn'] = self.create_lnet_residual

        self.params['train_ratio'] = {'lnet': 1}
        self.params['learning_rate'] = {'lnet': 1e-3}

        self.params['stats_mode'] = 'ema'
        self.params['stats_top_model_source'] = {
            'name': 'lnet',
            'node': 'errors',
            'target': 'sce-tumor',
            'key': 'topk'}

    def create_lnet_residual(self, inputs):

        nn_struct = {}

        # ------------------------------------------------
        # | FEATURE MAP DIMENSIONS|
        # L1: 120-120
        # L2: 060-060 
        # L3: 030-030 
        # L4: 015-015
        # L5: 007-007

        nn_struct['channels_out'] = [
            16, 16,
            32, 32,
            48, 48,
            64, 64,
            96, 96]

        nn_struct['filter_size'] = [
            [1, 7, 7], [1, 3, 3],
            [1, 3, 3], [1, 3, 3],
            [1, 3, 3], [1, 3, 3],
            [1, 3, 3], [1, 3, 3],
            [1, 3, 3], [1, 3, 3]]
        
        nn_struct['stride'] = [
            [1, 2, 2], 1,
            [1, 2, 2], 1,
            [1, 2, 2], 1,
            [1, 2, 2], 1,
            [1, 2, 2], 1]

        self.builder.create(
            name='L',
            nn_struct=nn_struct,
            input_layer=inputs[0])

        self.builder.graph.output['layer'] = self.flatten(self.builder.graph.output['layer'])
        self.create_logits(name='L')

In [ ]:
fdir = '/home/peter/caidm_brats/exps/classic/exp01'

net = cnn.Network()
net.Client = Client
net.Model = Model

net.initialize(
    app_context=app_context, 
    fold=0,
    threads=2, 
    batch=32,
    capacity=128
)

In [ ]:
net.train()

In [ ]:
viewer = cnn.Viewer('/home/peter/caidm_brats/exps/classic/exp01')
viewer.graph(name='lnet', node='errors', target='sce-tumor', key='topk')

In [ ]:
# Need kernel restart before running
def run_inference():

    client = Client(app_context=app_context)
    client.prepare(fold=0)

    inf = cnn.Inference(
        path='/home/peter/caidm_brats/exps/classic/exp01/final',
        client=client)
    
    results = inf.run_valid(n=1, draw=False)
    
    print(results['studyids'])
    print(results['logits'])

run_inference()